In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
interaction=pd.read_csv("../data/ratings.csv")
tag_interaction=pd.read_csv("../data/tags.csv")

In [3]:
tag_interaction.dropna(inplace=True)

In [4]:
user_movie_tag_data=pd.DataFrame(tag_interaction.groupby(["userId","movieId"])["tag"].apply(lambda x: "|".join(x)))
user_movie_tag_data

tag
userId movieId                                    
3      260                          classic|sci-fi
4      1732             dark comedy|great dialogue
       7569                       so bad it's good
       44665                  unreliable narrators
       115569                                tense
...                                            ...
162521 8874                       cornetto triolgy
       51255                      cornetto trilogy
       66934    Nathan Fillion|Neil Patrick Harris
       103341                     cornetto trilogy
162534 189169              comedy|disabled|robbery

[305341 rows x 1 columns]

In [5]:
user_movie_tag_with_ratings=user_movie_tag_data.merge(interaction,on=["userId","movieId"],how="left")

In [6]:
user_metadata=user_movie_tag_with_ratings[user_movie_tag_with_ratings["rating"]>=3][["userId","tag"]]

In [7]:
user_metadata=pd.DataFrame(user_metadata.groupby("userId")["tag"].apply(lambda x: "|".join(x)))

In [8]:
user_metadata.isnull().sum()

tag    0
dtype: int64

In [9]:
user_metadata.to_csv("../data/user_metadata.csv")